# Robustness Checks

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib as mpl 

mpl.rcParams['figure.dpi']= 200
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

from rpy2.robjects import pandas2ri
pandas2ri.activate()

%reload_ext rpy2.ipython

In [2]:
%%R

library('lme4')
library('margins')
library("performance")
library('tidyverse')
library('broom.mixed')

getICCs <- function(m, type = 'poisson'){
  var_s <- as.numeric(getME(m, "theta")[1]^2) # seller level variance
  var_w <- as.numeric(getME(m, "theta")[2]^2) # week level variance
  
  if(type == 'poisson'){
    lambda = .139
    alpha <- log(1 + 1/lambda)
  }
  if(type == 'binomial'){
    alpha <- (pi^2) / 3
  }
  
  icc <- list(
    s = (var_s)/ (var_s + var_w + alpha),
    w = (var_w)/ (var_s + var_w + alpha),
    t = (var_s + var_w)/ (var_s + var_w + alpha),
    a = (alpha)/ (var_s + var_w + alpha)
  )
  
  return(icc)
}

sessionInfo()

R[write to console]: Loading required package: Matrix

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

R[write to console]: Registered S3 method overwritten by 'broom.mixed':
  method      from 
  tidy.gamlss broom



R version 4.0.3 (2020-10-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur 10.16

Matrix products: default
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] broom.mixed_0.2.6 forcats_0.5.0     stringr_1.4.0     dplyr_1.0.2      
 [5] purrr_0.3.4       readr_1.4.0       tidyr_1.1.2       tibble_3.0.4     
 [9] ggplot2_3.3.2     tidyverse_1.3.0   performance_0.5.1 margins_0.3.23   
[13] lme4_1.1-25       Matrix_1.2-18    

loaded via a namespace (and not attached):
 [1] statmod_1.4.35    TMB_1.7.19        tidyselect_1.1.0  reshape2_1.4.4   
 [5] splines_4.0.3     haven_2.3.1       lattice_0.20-41   colorspace_1.4-1 
 [9] vctrs_0.3.4       generics_0.0.2    blob_1.2.1        rlang_0.4.8      
[13] nloptr_1.2.2.2    pillar_1.4.6      withr_2.3.0       glue_1.4.

## Load Data

In [3]:
MAINDIR = os.getcwd().rsplit('/', 1)[0]
file = os.path.join(MAINDIR, 'data/analysis', 'vendor_item_period.csv')

In [4]:
df = pd.read_csv(file, index_col = False)
df = df.drop(df.columns[0], axis=1)

In [5]:
me = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'market_entry.pickle'))

## Robustness of ARF

In our paper we used liberal cut of values, here we provide robustness checks for stricter cut of criteria

In [6]:
# setting cut of values
t_cut = me['delta_t'].quantile(1/3)
p_cut = me['price_usd'].quantile(1/3)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(1/3)


# ARF 1
arf1 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_025')

# setting cut of values
t_cut = me['delta_t'].quantile(2/5)
p_cut = me['price_usd'].quantile(2/5)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(2/5)


# ARF 1
arf2 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_041')

arf = pd.concat([arf1, arf2], axis = 1).reset_index()\
    .assign(vendor = lambda x: x['vendor'].factorize()[0] + 1)

## Robustness of ARM

In our paper we used global cut of values to determine cases of artificial growth. However, some sellers might aswell have show fast, static and cheap growth by definition. Therefore we perform robustness checks were we includes a stricter definition of ARM, including internally defined cut-off values. This assumes that ARM is a strategy that is used at some time-point but not all the time during market activity. However, it assures that the growth of reputation is relatively fast, static and cheap for sellers themselves. 

In [7]:
# make ARM variable
t_cut_per_seller = df.groupby('vendor')['delta_t_m'].quantile(1/2).to_dict()
s_cut_per_seller = df.groupby('vendor')['delta_t_s'].quantile(1/2).to_dict()
p_cut_per_seller = df.groupby('vendor')['price_usd_m'].quantile(1/2).to_dict()
t_cut = df['delta_t_m'].quantile(1/2)
p_cut = df['price_usd_m'].quantile(1/2)
s_cut = df['delta_t_s'].quantile(1/2)


arm = df\
    .assign(t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()

arm['arm_t__tps'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']


arm['arm_p__tps'] = df\
    .assign(p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['p_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_s__tps'] =  df\
    .assign(s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['s_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tp__tps'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_ts__tps'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 's_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_ps__tps'] = df\
    .assign(p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['p_low_s', 's_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_a'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__tp'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low', 'p_low', ]].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__ts'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__ps'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__p'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            p_low   = lambda x: x['price_usd_m'] < p_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 'p_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__t'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tps__s'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

# Shift Dependent variables with one week 
for col in ['arm', 'arm_t__tps', 'arm_p__tps', 'arm_s__tps',
            'arm_tp__tps', 'arm_ts__tps', 'arm_ps__tps', 
            'arm_a', 'arm_tps__tp', 'arm_tps__ts', 
            'arm_tps__ps', 'arm_tps__p', 'arm_tps__t', 'arm_tps__s']: 
    arm['{0}_shift'.format(col)] = arm\
        .groupby('vendor')\
        .apply(lambda x: x[col].shift(1))\
        .reset_index(drop = True)

In [8]:
# import person week file
df = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'vendor_week.pickle'))
df = df.merge(df.groupby('vendor')['me_min'].min().rename('me'), on = 'vendor')
df = df.rename(columns= {
    'international_shipment_count_w': 'int_shipment_count_w', 
    'international_shipment': 'int_shipment'})

In [9]:
df = df.merge(arm, on = ['vendor', 'w']).merge(arf, on = ['vendor'])

In [10]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

# normalize variables
skewed_variables = [
    'neg_count_min', 'pos_count_min', 'neg_count_w', 
    'neg_count_w_shift', 'pos_count_w', 'pos_count_w_shift', 'sales_volume_w', 
    'item_count_w', 'int_shipment_count_w']

bc_vars = pd.DataFrame(
    pt.fit_transform(df[skewed_variables]),
    columns = ['bc_' + var for var in skewed_variables])

c_vars = df[skewed_variables].add_prefix('c_')

dummy_vars = df.assign(
        items   = pd.qcut(df['item_count'], 3,  labels=["low", "medium", "high"]),
        items_w = pd.qcut(df['item_count_w'], 3,  labels=["low", "medium", "high"]),
        sales   = pd.qcut(df['cum_count'], 2,  labels=["low", "high"]),
        sales_w = pd.qcut(df['sales_volume_w'], 2,  labels=["low", "high"]))\
    .loc[:,['items', 'items_w', 'sales', 'sales_w']]

n_vars = df[['cum_count', 'sales_volume_w', 'item_count', 
             'item_count_w', 'me', 'int_shipment_count_w',
             'empty_stock_last_week_count_w', 'empty_stock_last_week_maxw']].rename({'cum_count': 'sales_volume'})

bool_vars = df[
    ['arf', 'arf_025', 'arf_041', 'arm_maxw', 'arm_maxw_shift', 
     'arm', 'arm_t__tps', 'arm_p__tps', 'arm_s__tps',
     'arm_tp__tps', 'arm_ts__tps', 'arm_ps__tps', 
     'arm_a', 'arm_tps__tp', 'arm_tps__ts', 
     'arm_tps__ps', 'arm_tps__p', 'arm_tps__t', 'arm_tps__s', 
     'int_shipment', 'has_price_drop', 'has_price_drop_shift']].astype('int')

lev_vars =  df[['vendor','w', 'me_min']]

data = pd.concat([lev_vars, bool_vars, bc_vars, c_vars, dummy_vars, n_vars], axis = 1)

In [11]:
%%R -i data

data['me_'] <- scale(data$me)[,1]
data['w_'] <- scale(data$w)[,1]
data['me_2'] <- scale(data$me^2)[,1]
data['w_2'] <- scale(data$w^2)[,1]

variables = c(
    'bc_pos_count_w', 'bc_item_count_w', 'bc_int_shipment_count_w',
    'bc_neg_count_w', 'c_neg_count_w', 'c_pos_count_w')

for (var in variables){
  for (vendor in unique(data$vendor)){
    # calculate mean and deviances
    x = data[data$vendor == vendor, var]
    m = mean(x)
    dev = x - m
    
    # concat information to dataframe
    data[data$vendor == vendor, paste(var, "m", sep=".")] <- m
    data[data$vendor == vendor, paste(var, "dev", sep=".")] <- dev
  }
}

Optimizer <- glmerControl(optimizer = "bobyqa",
                          optCtrl = list(maxfun=2e5))

model_arf <- list()

## ARF
### Stricter criteria

In [12]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a114 = model))

In [13]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a041 = model))

In [14]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a025 = model));

In [15]:
%%R
model <- glmer(arm ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a114 = model))

In [16]:
%%R
model <- glmer(arm ~ arf_041
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_041
      + bc_neg_count_w.dev:arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a041 = model))

In [17]:
%%R
model <- glmer(arm ~ arf_025
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_025
      + bc_neg_count_w.dev:arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a025 = model))

In [20]:
%%R -o results

cols = c("B", 'se', 'z', 'p')

results = data.frame()

for (name in names(model_arf)){
    model <- round(data.frame(summary(model_arf[[name]])[10]),3)
    colnames(model) <- cols
    rownames(model) <- paste(name, rownames(model), sep = '__')
    
    odds <- data.frame(tidy(model_arf[[name]],conf.int=TRUE,exponentiate=TRUE,effects="fixed")) %>%
        select(c("estimate", "conf.low", "conf.high")) %>%
        mutate_all(funs(round(.,3)))
    colnames(odds) <- c("or", "ci_low", "ci_high")
    
    model <- cbind(model, odds)
    
    results <- rbind(results, model)
}

In [65]:
def sig_stars(x):
    if x < .001: 
        return "***"
    if x < .01: 
        return "**"
    if x < .05:
        return "*"
    if x < .1: 
        return "."
    else: 
        return ""

ci_fmt = lambda x: f"({str(round(x['ci_low'], 3)).ljust(5, '0')}; {str(round(x['ci_high'], 3)).ljust(5, '0')})"
order = [0, 1] + [12, 13, 14, 15] + [ 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
col_order = [5, 4, 3, 2, 1, 0]

r = results\
    .assign(star = lambda x: x['p'].apply(sig_stars))\
    .assign(_or = lambda x: x["or"].apply(lambda v: f'{v:.3f}') + x["star"],
            ci = lambda row: row.agg(ci_fmt, axis=1))[['_or', 'ci']]\
    .reset_index()\
    .melt(id_vars = 'index')\
    .assign(
        condition = lambda x: x['index'].str.split('__').str[0],
        coefficient = lambda x: x['index'].str.split('__').str[1]\
                                          .str.replace('_041', '')\
                                          .str.replace('_025', ''))\
    .assign(
        coefficient = lambda x: pd.Categorical(x['coefficient'], 
                                               ordered=True, 
                                               categories=[x['coefficient'].unique()[i] for i in order]))\
    .drop('index', axis = 1)\
    .set_index(['coefficient', 'variable', 'condition'])\
    .unstack()[lambda x: [x.columns[i] for i in col_order]]

r

value                     \
condition                                      neg_a114           neg_a041   
coefficient                 variable                                         
(Intercept)                 _or                 0.631**            0.660**   
                            ci           (0.470; 0.848)     (0.493; 0.885)   
arf                         _or                 1.606**             2.008*   
                            ci           (1.194; 2.160)     (1.076; 3.749)   
bc_neg_count_w.m            _or                     NaN                NaN   
                            ci                      NaN                NaN   
bc_neg_count_w.dev          _or                     NaN                NaN   
                            ci                      NaN                NaN   
arf:bc_neg_count_w.m        _or                     NaN                NaN   
                            ci                      NaN                NaN   
arf:bc_neg_count_w.dev      _or                     NaN                NaN   
                            ci                      NaN                NaN   
bc_pos_count_w.m            _or                   1.186             1.240.   
                            ci           (0.954; 1.475)     (0.999; 1.538)   
bc_pos_count_w.dev          _or                1.043***           1.043***   
                            ci           (1.032; 1.054)     (1.032; 1.054)   
bc_item_count_w.m           _or                1.847***           1.766***   
                            ci           (1.479; 2.306)     (1.414; 2.204)   
bc_item_count_w.dev         _or                0.848***           0.848***   
                            ci           (0.826; 0.869)     (0.826; 0.869)   
bc_int_shipment_count_w.m   _or                1.498***           1.515***   
                            ci           (1.319; 1.702)     (1.333; 1.722)   
bc_int_shipment_count_w.dev _or                1.137***           1.137***   
                            ci           (1.091; 1.185)     (1.091; 1.185)   
me_                         _or                  1.293.             1.282.   
                            ci           (0.998; 1.675)     (0.989; 1.662)   
me_2                        _or                0.638***           0.639***   
                            ci           (0.503; 0.809)     (0.504; 0.811)   
w_                          _or               54.618***          54.582***   
                            ci        (18.842; 158.322)  (19.362; 153.866)   
w_2                         _or                0.006***           0.006***   
                            ci           (0.002; 0.018)     (0.002; 0.017)   

                                                                            \
condition                                      neg_a025           arm_a114   
coefficient                 variable                                         
(Intercept)                 _or                 0.673**           0.002***   
                            ci           (0.502; 0.901)     (0.002; 0.003)   
arf                         _or                   1.069            1.923**   
                            ci           (0.463; 2.472)     (1.199; 3.084)   
bc_neg_count_w.m            _or                     NaN           2.433***   
                            ci                      NaN     (1.840; 3.218)   
bc_neg_count_w.dev          _or                     NaN              1.010   
                            ci                      NaN     (0.930; 1.097)   
arf:bc_neg_count_w.m        _or                     NaN              1.007   
                            ci                      NaN     (0.522; 1.939)   
arf:bc_neg_count_w.dev      _or                     NaN              1.054   
                            ci                      NaN     (0.882; 1.259)   
bc_pos_count_w.m            _or                  1.254*          21.737***   
                            ci           (1.010; 1.557)   (14.566; 32.4

## ARM
### Internal and external cut-offs

In [95]:
%%R

model_arm <- list()

model <- glmer(arm ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm = model))

In [96]:
%%R

model <- glmer(arm_t__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_t__tps = model))

In [97]:
%%R

model <- glmer(arm_p__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_p__tps = model))

In [98]:
%%R

model <- glmer(arm_s__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_s__tps = model))

In [99]:
%%R
model <- glmer(arm_tp__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tp__tps = model))

In [100]:
%%R
model <- glmer(arm_ts__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_ts__tps = model))

In [101]:
%%R
model <- glmer(arm_ps__tps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_ps__tps = model))

In [102]:
%%R
model <- glmer(arm_a ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_a = model))

In [103]:
%%R
model <- glmer(arm_tps__tp ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__tp = model))

In [104]:
%%R
model <- glmer(arm_tps__ts ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__ts = model))

In [105]:
%%R
model <- glmer(arm_tps__ps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__ps = model))

In [106]:
%%R
model <- glmer(arm_tps__t ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__t = model))

In [107]:
%%R
model <- glmer(arm_tps__p ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__p = model))

In [108]:
%%R
model <- glmer(arm_tps__s ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tps__s = model))

In [119]:
%%R -o results2

cols <- c("B", 'se', 'z', 'p')

results2 <- data.frame()

for (name in names(model_arm)){
    model <- round(data.frame(summary(model_arm[[name]])[10]),3)
    colnames(model) <- cols
    rownames(model) <- paste(name, rownames(model), sep = '___')

    odds <- data.frame(tidy(model_arm[[name]],conf.int=TRUE,exponentiate=TRUE,effects="fixed")) %>%
        select(c("estimate", "conf.low", "conf.high")) %>%
        mutate_all(funs(round(.,3)))
    colnames(odds) <- c("or", "ci_low", "ci_high")
    
    model <- cbind(model, odds)
     
    results2 <- rbind(results2, model)
}


In [120]:
results2

,B,se,z,p,or,ci_low,ci_high
arm___(Intercept),-6.036,0.151,-39.920,0.000,0.002,0.002,0.003
arm___arf,0.654,0.241,2.713,0.007,1.923,1.199,3.084
arm___bc_neg_count_w.m,0.889,0.143,6.233,0.000,2.433,1.840,3.218
arm___bc_neg_count_w.dev,0.010,0.042,0.240,0.810,1.010,0.930,1.097
arm___bc_pos_count_w.m,3.079,0.204,15.074,0.000,21.737,14.566,32.439
...,...,...,...,...,...,...,...
arm_tps__s___me_2,0.197,0.096,2.045,0.041,1.218,1.008,1.471
arm_tps__s___w_,-0.068,0.092,-0.739,0.460,0.934,0.780,1.119
arm_tps__s___w_2,0.248,0.091,2.718,0.007,1.281,1.072,1.532
arm_tps__s___arf:bc_neg_count_w.m,-0.156,0.181,-0.864,0.387,0.855,0.600,1.219


In [139]:
def sig_stars(x):
    if x < .001: 
        return "***"
    if x < .01: 
        return "**"
    if x < .05:
        return "*"
    if x < .1: 
        return "."
    else: 
        return ""

order = [0, 1, 2, 3, 4]

results2\
    .assign(star = lambda x: x['p'].apply(sig_stars))\
    .assign(_or = lambda x: x["or"].apply(lambda v: f'{v:.3f}') + x["star"],
            ci = lambda row: row.agg(ci_fmt, axis=1))[['_or', 'ci']]\
    .reset_index()\
    .melt(id_vars = 'index')\
    .assign(
        condition = lambda x: x['index'].str.split('___').str[0],
        coefficient = lambda x: x['index'].str.split('___').str[1]\
                                          .str.replace('_041', '')\
                                          .str.replace('_025', ''))\
    .drop('index', axis = 1)\
    .loc[lambda x: (x['coefficient'].str.contains('arf')) | (x['coefficient'].str.contains('neg_count'))]\
    .assign(
        coefficient = lambda x: pd.Categorical(x['coefficient'], 
                                               ordered=True, 
                                               categories=[x['coefficient'].unique()[i] for i in order]))\
    .sort_values('coefficient')\
    .pivot_table(index= 'coefficient', columns= ['condition', 'variable'], values = 'value', aggfunc='max').T

coefficient                      arf bc_neg_count_w.m bc_neg_count_w.dev  \
condition   variable                                                       
arm         _or              1.923**         2.433***              1.010   
            ci        (1.199; 3.084)   (1.840; 3.218)     (0.930; 1.097)   
arm_a       _or                1.481         2.442***              0.990   
            ci        (0.893; 2.456)   (1.814; 3.288)     (0.915; 1.071)   
arm_p__tps  _or               1.636*         2.284***              0.990   
            ci        (1.035; 2.584)   (1.749; 2.984)     (0.910; 1.076)   
arm_ps__tps _or               1.663*         2.391***              0.977   
            ci        (1.049; 2.638)   (1.823; 3.135)     (0.900; 1.061)   
arm_s__tps  _or              2.038**         2.511***              1.000   
            ci        (1.265; 3.284)   (1.889; 3.339)     (0.921; 1.085)   
arm_t__tps  _or                0.963         1.911***              1.007   
            ci        (0.596; 1.555)   (1.449; 2.520)     (0.935; 1.085)   
arm_tp__tps _or                1.423         2.306***              0.990   
            ci        (0.876; 2.312)   (1.737; 3.062)     (0.915; 1.071)   
arm_tps__p  _or               1.388.         1.808***              1.010   
            ci        (0.975; 1.977)   (1.465; 2.233)     (0.961; 1.061)   
arm_tps__ps _or               1.341.         1.685***              1.030   
            ci        (0.985; 1.826)   (1.401; 2.027)     (0.977; 1.086)   
arm_tps__s  _or                0.944          1.297**              1.026   
            ci        (0.740; 1.205)   (1.120; 1.502)     (0.976; 1.080)   
arm_tps__t  _or                0.963         1.911***              1.007   
            ci        (0.596; 1.555)   (1.449; 2.520)     (0.935; 1.085)   
arm_tps__tp _or                1.332         2.485***              1.006   
            ci        (0.764; 2.323)   (1.798; 3.434)     (0.933; 1.085)   
arm_tps__ts _or                1.051         1.823***              0.999   
            ci        (0.678; 1.630)   (1.412; 2.353)     (0.925; 1.080)   
arm_ts__tps _or               1.592.         2.611***              1.011   
            ci        (0.947; 2.674)   (1.920; 3.551)     (0.935; 1.093)   

coefficient          arf:bc_neg_count_w.m arf:bc_neg_count_w.dev  
condition   variable                                              
arm         _or                     1.007                  1.054  
            ci             (0.522; 1.939)         (0.882; 1.259)  
arm_a       _or                     0.693                  1.039  
            ci             (0.345; 1.390)         (0.882; 1.223)  
arm_p__tps  _or                     1.050                 1.175.  
            ci             (0.557; 1.980)         (0.982; 1.405)  
arm_ps__tps _or                     1.015                  1.156  
            ci             (0.536; 1.924)         (0.969; 1.379)  
arm_s__tps  _or                     0.965                  1.053  
            ci             (0.497; 1.871)         (0.883; 1.256)  
arm_t__tps  _or                     0.968                  0.992  
            ci             (0.501; 1.873)         (0.850; 1.157)  
arm_tp__tps _or                     0.757                  1.056  
            ci             (0.388; 1.477)         (0.896; 1.244)  
arm_tps__p  _or                    0.526*                  0.996  
            ci             (0.315; 0.878)         (0.888; 1.118)  
arm_tps__ps _or                    0.648.                  0.984  
            ci             (0.415; 1.013)         (0.869; 1.115)  
arm_tps__s  _or                     0.855                  0.989  
            ci             (0.600; 1.219)         (0.875; 1.117)  
arm_tps__t  _or                     0.968                  0.992  
            ci             (0.501; 1.873)         (0.850; 1.157)  
arm_tps__tp _or                     0.832                  1.001  
            ci             (0.387; 1.788)   